In [7]:
import pandas as pd
import numpy as np

# Load the uploaded dataset to preview its structure
file_path = 'https://raw.githubusercontent.com/qmdismnp/Schulich_DS_MBAN/refs/heads/main/data_set_hackathon.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data


,order_date,requested_delivery_date,Customer Country Code,Product Code,Description,order_type,Customer Order Code,value,Curr,items,Route
0,13.07.2009,28.01.2010,RU,L10705000,Parka Outdoor Lifestyle STD,VO,3200435553,2337.00,RUB,6,RU0001
1,15.07.2009,24.03.2010,RU,L10705000,Parka Outdoor Lifestyle STD,VO,3200435694,10160.25,RUB,23,RU0001
2,16.07.2009,04.02.2010,RU,L10705000,Parka Outdoor Lifestyle STD,VO,3200435741,2992.50,RUB,7,RU0001
3,17.07.2009,04.02.2010,RU,L10705000,Parka Outdoor Lifestyle STD,VO,3200435907,4061.25,RUB,9,RU0001
4,21.07.2009,01.02.2010,RU,L10705000,Parka Outdoor Lifestyle STD,VO,3200435963,2208.75,RUB,5,RU0001
...,...,...,...,...,...,...,...,...,...,...,...
2415,13.07.2011,15.02.2012,HR,L12919200,Parka Outdoor Lifestyle STD,VO,3200819196,128.52,EUR,12,FI0003
2416,13.07.2011,15.02.2012,HR,L12919200,Parka Outdoor Lifestyle STD,VO,3200819201,128.52,EUR,12,FI0003
2417,13.07.2011,15.02.2012,HR,L12919200,Parka Outdoor Lifestyle STD,VO,3200819206,128.52,EUR,12,FI0003
2418,13.07.2011,15.02.2012,HR,L12919200,Parka Outdoor Lifestyle STD,VO,3200819210,107.10,EUR,10,FI0003


In [2]:
# Handle missing or invalid values
data.replace(r'\\N', np.nan, regex=True, inplace=True)  # Replace invalid strings
data.fillna(0, inplace=True)  # Replace NaN values with 0 (or use appropriate imputation)

In [6]:
# Convert the "Order Date" column to datetime format
data['order_date'] = pd.to_datetime(data['order_date'], format='%d.%m.%Y')
data['requested_delivery_date'] = pd.to_datetime(data['requested_delivery_date'], format='%d.%m.%Y')

# Extract the year and month from the "Order Date"
data['Year-Month'] = data['order_date'].dt.to_period('M')

# Group by the Year-Month and count the distinct "Customer Order Code"
monthly_order_counts = data.groupby('Year-Month')['Customer Order Code'].nunique().reset_index()

# Rename columns for clarity
monthly_order_counts.columns = ['Year-Month', 'Distinct Orders']

# Convert Year-Month to string format for easier readability
monthly_order_counts['Year-Month'] = monthly_order_counts['Year-Month'].astype(str)

monthly_order_counts

,Year-Month,Distinct Orders
0,2009-07,38
1,2009-08,9
2,2009-09,12
3,2009-10,4
4,2009-11,2
5,2009-12,21
6,2010-01,29
7,2010-02,36
8,2010-03,43
9,2010-04,11


In [3]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

def calculate_monthly_orders_with_sarima(data):
    """
    Groups transactional data by month to calculate the number of unique orders.
    Applies SARIMA model for forecasting and evaluates its performance.
    Forecasts for the next five months and the next two months.
    """
    # Step 1: Preprocessing
    data['order_date'] = pd.to_datetime(data['order_date'], format='%d.%m.%Y')
    data['year_month'] = data['order_date'].dt.to_period('M')
    monthly_orders = (
        data.groupby('year_month')['Customer Order Code']
        .nunique()
        .reset_index(name='distinct_orders')
    )

    # Prepare data for SARIMA
    monthly_orders['year_month'] = pd.to_datetime(monthly_orders['year_month'].astype(str))
    monthly_orders.set_index('year_month', inplace=True)

    # Split data into training and testing sets
    train_size = int(len(monthly_orders) * 0.8)
    train_data = monthly_orders.iloc[:train_size]
    test_data = monthly_orders.iloc[train_size:]

    # Step 2: Fit SARIMA model
    sarima_model = SARIMAX(train_data['distinct_orders'],
                           order=(1, 1, 1),
                           seasonal_order=(1, 1, 1, 12))
    sarima_result = sarima_model.fit(disp=False)

    # Step 3: Forecast for the test set (existing data)
    forecast_test = sarima_result.forecast(steps=len(test_data))
    mape_test = mean_absolute_percentage_error(test_data['distinct_orders'], forecast_test)

    # Step 4: Forecast for future months
    future_forecast_5_months = sarima_result.forecast(steps=5)
    future_forecast_2_months = sarima_result.forecast(steps=2)

    return future_forecast_5_months, future_forecast_2_months, mape_test


In [122]:
calculate_monthly_orders_with_sarima(data)

/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non

/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Users/saki/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get

(21    100.749159
 22     75.257952
 23     80.277692
 24    144.551800
 25    126.099667
 Name: predicted_mean, dtype: float64,
 21    100.749159
 22     75.257952
 Name: predicted_mean, dtype: float64,
 70.12237951089121)

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def classify_and_evaluate_product_demand(data, months=5):
    """
    Prepares data for a classification model by encoding features, trains a logistic regression model,
    and evaluates it. Forecasts demand for the next specified number of months.
    """
    # Add seasonality based on the order date
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'

    data['order_date'] = pd.to_datetime(data['order_date'], format='%d.%m.%Y')
    data['Season'] = data['order_date'].dt.month.apply(get_season)

    # Encode categorical variables
    encoded_data = pd.get_dummies(data, columns=['Season', 'Customer Country Code','Curr', 'Route', 'order_type'], drop_first=True)

    # Define features and target variable
    X = encoded_data.drop(columns=['Product Code', 'year_month', 'order_date', 'requested_delivery_date', 'Customer Order Code', 'Description'])
    y = encoded_data['Product Code']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a logistic regression model
    logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
    logistic_model.fit(X_train, y_train)

    # Make predictions
    y_pred_logistic = logistic_model.predict(X_test)

    # Forecast demand for the next `months`
    future_forecast_demand = logistic_model.predict(X_test.sample(n=months, random_state=42))

    return y_pred_logistic,future_forecast_demand


In [124]:
classify_and_evaluate_product_demand(data)

/Users/saki/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(array(['L12135800', 'L12918000', 'L12918400', 'L12916900', 'L12918700',
        'L12135800', 'L12916900', 'L12916900', 'L12916900', 'L12918000',
        'L12916900', 'L12918000', 'L12918400', 'L12916900', 'L12916900',
        'L12916900', 'L12918000', 'L12918000', 'L12918400', 'L12916900',
        'L12919200', 'L12918400', 'L12918000', 'L12916900', 'L12916900',
        'L12918400', 'L12918000', 'L12918000', 'L12135800', 'L12916900',
        'L12918000', 'L12918400', 'L12135800', 'L12918400', 'L12918700',
        'L12918400', 'L12916900', 'L12918000', 'L12918000', 'L12916900',
        'L12918400', 'L12916900', 'L12916900', 'L12135800', 'L12916900',
        'L12918000', 'L12918400', 'L12918000', 'L12918000', 'L12918700',
        'L12918000', 'L12916900', 'L12918400', 'L12918700', 'L12916900',
        'L12918400', 'L12918000', 'L12918400', 'L12916900', 'L12135800',
        'L12916900', 'L12918000', 'L12918400', 'L12135800', 'L12916900',
        'L12918400', 'L12918400', 'L12918000', 'L12

In [125]:
# Ensure the 'items' column is numeric
data['items'] = pd.to_numeric(data['items'], errors='coerce')

# Drop rows with NaN in the 'items' column after conversion
data = data.dropna(subset=['items'])

# Group by 'Product Code' and calculate the quantiles
quantity_bounds = data.groupby('Product Code')['items'].quantile([0.25, 0.75]).unstack().reset_index()

# Rename the columns for clarity
quantity_bounds.columns = ['Product Code', '25th Percentile', '75th Percentile']


In [126]:
import pandas as pd
import numpy as np

def simulate_quantity_demand(data, n_months=5):
    """
    Recalculates 25th and 75th percentiles for each product and simulates demand for the next n months.
    :param data: Pandas DataFrame with columns 'Product Code' and 'items'.
    :param n_months: Number of months to simulate demand for.
    :return: DataFrame with simulated demand for each product.
    """
    # Recalculate percentiles for each product
    # Ensure the 'items' column is numeric
    data['items'] = pd.to_numeric(data['items'], errors='coerce')
    # Drop rows with NaN in the 'items' column after conversion
    data = data.dropna(subset=['items'])
    # Group by 'Product Code' and calculate the quantiles
    quantity_bounds = data.groupby('Product Code')['items'].quantile([0.25, 0.75]).unstack().reset_index()
    # Rename the columns for clarity
    quantity_bounds.columns = ['Product Code', '25th Percentile', '75th Percentile']


    # Simulate demand for the next n_months
    simulated_demand = []

    for _, row in quantity_bounds.iterrows():
        mean_quantity = (row['25th Percentile'] + row['75th Percentile']) / 2
        std_dev_quantity = (row['75th Percentile'] - row['25th Percentile']) / 6  # Assuming normal distribution
        product_demand = np.random.normal(mean_quantity, std_dev_quantity, n_months).clip(0)  # Ensure no negative values
        simulated_demand.append(product_demand)

    # Create a DataFrame for the simulated demand
    simulated_demand_data = pd.DataFrame(
        simulated_demand,
        columns=[f"Month {i+1}" for i in range(n_months)],
        index=quantity_bounds['Product Code']
    ).reset_index()

    # Rename columns for clarity
    simulated_demand_data.rename(columns={'index': 'Product Code'}, inplace=True)

    return simulated_demand_data


In [127]:
simulate_quantity_demand(data)

,Product Code,Month 1,Month 2,Month 3,Month 4,Month 5
0,L10705000,6.539206,6.067611,5.990160,5.868522,7.189895
1,L10705100,10.415796,9.695894,8.634418,9.281492,9.184452
2,L10705200,7.659447,6.550032,5.828140,7.807474,7.709865
3,L10705300,7.457564,8.341073,7.140094,8.653944,8.248016
4,L10705400,3.258388,3.371500,6.514450,8.470349,4.686003
5,L10705500,11.745298,7.635473,8.700920,9.377020,8.703402
6,L10705600,5.509635,7.120167,6.985174,6.534129,6.177324
7,L10705700,4.408034,6.196618,4.841986,4.856045,5.449505
8,L10850600,5.757806,7.627138,6.604239,7.476129,4.964017
9,L10850700,2.109718,1.213663,3.256133,3.393926,2.053423


In [132]:
import pandas as pd
import numpy as np

def analyze_and_simulate_lead_time(data, n_simulations=1000):
    """
    Calculates lead time, analyzes its distribution, and simulates lead times based on observed data.
    :param data: Pandas DataFrame with 'order_date' and 'requested_delivery_date' columns.
    :param n_simulations: Number of simulated lead time samples.
    :return: Dictionary containing observed quantiles and simulated lead times.
    """
    # Preprocess data: calculate lead time
    data['order_date'] = pd.to_datetime(data['order_date'], format='%d.%m.%Y')
    data['requested_delivery_date'] = pd.to_datetime(data['requested_delivery_date'], format='%d.%m.%Y')
    data['lead_time'] = (data['requested_delivery_date'] - data['order_date']).dt.days
    
    # Analyze lead time distribution
    lead_time_quantiles = data['lead_time'].quantile([0.05, 0.5, 0.95])
    
    # Simulate lead times
    mean_lead_time = lead_time_quantiles[0.5]
    std_dev_lead_time = (lead_time_quantiles[0.95] - lead_time_quantiles[0.05]) / 1.35  # Approximation using IQR
    simulated_lead_times = np.random.normal(mean_lead_time, std_dev_lead_time, n_simulations).clip(0)
    
    # Consolidate results
    result = {
        'observed_quantiles': lead_time_quantiles,
        'simulated_lead_times': simulated_lead_times
    }
    return result


In [133]:
analyze_and_simulate_lead_time(data)

{'observed_quantiles': 0.05    140.95
 0.50    217.00
 0.95    312.00
 Name: lead_time, dtype: float64,
 'simulated_lead_times': array([ 44.1250531 , 284.58643331, 252.34764055, 349.32421098,
        332.88862217,  35.60688955, 325.80920143, 338.13952514,
        414.59361431, 417.71553425, 208.39611091, 137.54389121,
        253.24389139, 148.88352749, 126.05777901,  96.70636753,
        380.6562785 , 253.55980103, 146.32323533, 224.66941817,
        145.46870984, 525.53737536, 393.57002872, 129.63628036,
        141.61175972, 199.84892507,  97.93711433,   0.        ,
        224.31651098, 282.22476683, 202.65153914, 218.75293349,
        134.06280649,   0.        , 307.61161095, 376.46491909,
        384.08637432, 110.99274542, 304.61036315, 459.63917564,
        161.99785566, 341.96962268, 251.3175236 , 139.42955378,
        217.37615345, 295.33927891, 140.76419661, 340.48440189,
        334.6080357 ,  90.95289608, 307.12225961, 108.70152868,
        172.07044476, 142.12561365,  51.

In [129]:
import numpy as np

def monte_carlo_simulation(demand_bounds, n_simulations=1000):
    """
    Simulates demand using Monte Carlo for a given number of iterations.
    :param demand_bounds: DataFrame with 25th and 75th percentiles of demand.
    :param n_simulations: Number of Monte Carlo iterations.
    :return: Simulated demand DataFrame.
    """
    simulated_demand = []
    for _, row in demand_bounds.iterrows():
        mean_quantity = (row['25th Percentile'] + row['75th Percentile']) / 2
        std_dev_quantity = (row['75th Percentile'] - row['25th Percentile']) / 6  # Approximation
        product_demand = np.random.normal(mean_quantity, std_dev_quantity, n_simulations).clip(0)
        simulated_demand.append(product_demand)
    return np.array(simulated_demand)


In [130]:
def monte_carlo_total_demand_simulation(data, n_simulations=1000, lead_time_threshold=30):
    """
    Performs Monte Carlo simulation to consolidate uncertainties into total demand,
    distinguishing advance demand from urgent demand.
    """
    # Step 1: Prepare data
    quantity_bounds = data.groupby('Product Code')['items'].quantile([0.25, 0.75]).unstack()
    quantity_bounds.columns = ['25th Percentile', '75th Percentile']
    lead_time_stats = data['lead_time'].quantile([0.25, 0.5, 0.75])  # Include 0.5 quantile explicitly
    
    mean_lead_time = lead_time_stats[0.5]  # Use median for lead time mean
    std_dev_lead_time = (lead_time_stats[0.75] - lead_time_stats[0.25]) / 1.35  # Approximation using IQR

    # Step 2: Monte Carlo simulation
    simulated_demand = []
    for _ in range(n_simulations):
        scenario_demand = []
        for _, row in quantity_bounds.iterrows():
            mean_quantity = (row['25th Percentile'] + row['75th Percentile']) / 2
            std_dev_quantity = (row['75th Percentile'] - row['25th Percentile']) / 6
            quantity = np.random.normal(mean_quantity, std_dev_quantity).clip(0)
            lead_time = np.random.normal(mean_lead_time, std_dev_lead_time).clip(0)
            scenario_demand.append((quantity, lead_time))
        simulated_demand.append(scenario_demand)

    # Step 3: Analyze simulated data
    advance_demand = []
    urgent_demand = []
    for scenario in simulated_demand:
        advance = sum(q for q, lt in scenario if lt > lead_time_threshold)
        urgent = sum(q for q, lt in scenario if lt <= lead_time_threshold)
        advance_demand.append(advance)
        urgent_demand.append(urgent)

    # Step 4: Consolidate results
    results = {
        'simulated_advance_demand': np.array(advance_demand),
        'simulated_urgent_demand': np.array(urgent_demand),
        'summary_stats': {
            'advance_mean': np.mean(advance_demand),
            'urgent_mean': np.mean(urgent_demand),
            'advance_std': np.std(advance_demand),
            'urgent_std': np.std(urgent_demand)
        }
    }
    return results



In [131]:
# Perform Monte Carlo simulation with corrected function
simulation_results = monte_carlo_total_demand_simulation(data, n_simulations=10, lead_time_threshold=30)

# Analyze results
simulation_results_summary = {
    "Advance Demand Mean": simulation_results['summary_stats']['advance_mean'],
    "Urgent Demand Mean": simulation_results['summary_stats']['urgent_mean'],
    "Advance Demand Std Dev": simulation_results['summary_stats']['advance_std'],
    "Urgent Demand Std Dev": simulation_results['summary_stats']['urgent_std']
}

simulation_results_summary


KeyError: 'lead_time'